In [1]:
# Importando Bibliotecas
import requests
import re
import time
import json
from bs4 import BeautifulSoup
import google.generativeai as genai
import os
from dotenv import load_dotenv
import google.generativeai as genai

c:\Users\PedroLustosa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# URL
diseases_url = "https://www.who.int/news-room/fact-sheets"

# Requisição
try:
    response = requests.get(diseases_url)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    print('Erro ao fazer a requisição:', e)
    exit()

diseases_html = BeautifulSoup(response.content, 'html.parser')

In [3]:
# Encontrar todos os elementos com a classe "item"
items = diseases_html.find_all('p')

# Inicializar uma lista para armazenar as doenças
diseases = []

# Exibir os textos dos elementos <a> dentro dos elementos com a classe "item"
for item in items:
        disease = item.text
        diseases.append(disease)



In [4]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Acessa a chave de API do Gemini
GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

# Define o modelo
generation_config = {
  "temperature": 0.3,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

instruction = """You are an expert medical professional, equipped with comprehensive knowledge of human diseases and their symptoms
You create correct and simple symptoms list that is easy to understand.
Without title
Without comments
"""

model = genai.GenerativeModel(model_name="gemini-1.5-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings,
                              system_instruction=instruction)

In [5]:
# Inicialização de dicionário para armazenar sintomas das doenças e lista para erros
disease_symptoms = {}  # Dicionário para armazenar os sintomas associados a cada doença
disease_error = []  # Lista para armazenar erros ocorridos durante o processamento das doenças


In [6]:
def fetch_disease_info(disease):
    """
    Função para obter informações de uma doença a partir da URL da WHO.
    
    Parâmetros:
    disease (str): Nome da doença
    
    Retorna:
    response (requests.Response): Resposta da requisição HTTP
    ou
    None: Se ocorrer um erro na requisição
    """
    # Limpa e formata o nome da doença para criar a URL
    # Substitui espaços por hífens e remove caracteres não alfanuméricos e hífens
    disease_clear = re.sub(r"[^a-z0-9-]", "", disease.lower().replace(" ", "-"))
    disease_html = f'https://www.who.int/news-room/fact-sheets/detail/{disease_clear}'

    try:
        # Faz uma requisição HTTP para obter o conteúdo da página da doença
        response = requests.get(disease_html)
        # Verifica se a requisição foi bem-sucedida
        response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
    except requests.exceptions.RequestException as e:
        # Em caso de erro na requisição, adiciona o erro à lista e retorna None
        disease_error.append(e)
        return None
    
    # Aguarda 1 segundo para evitar sobrecarga no servidor
    time.sleep(1)
    return response


In [7]:
def parse_symptoms(response_content):
    """
    Função para analisar o conteúdo HTML e extrair os sintomas.

    Parâmetros:
    response_content (str): Conteúdo HTML da página da doença

    Retorna:
    symptoms (str): String contendo os sintomas extraídos
    """
    # Cria um objeto BeautifulSoup para analisar o conteúdo HTML
    soup = BeautifulSoup(response_content, 'html.parser')

    # Procura um cabeçalho <h2> que contenha as palavras 'consequences' ou 'symptoms'
    symptoms_header = soup.find('h2', string=re.compile(r'consequences|symptoms', re.IGNORECASE))
    
    # Se não encontrar um cabeçalho de sintomas, retorna uma string vazia
    if not symptoms_header:
        return ''

    # Inicializa uma string vazia para armazenar os sintomas
    symptoms = ''
    
    # Encontra o próximo elemento após o cabeçalho de sintomas
    current_element = symptoms_header.find_next_sibling()
    
    # Itera sobre os elementos subsequentes até encontrar outro cabeçalho <h2> ou o fim do documento
    while current_element and current_element.name != 'h2':
        # Adiciona o texto do elemento atual à string de sintomas
        symptoms += current_element.text
        # Passa para o próximo elemento
        current_element = current_element.find_next_sibling()
    
    # Retorna a string contendo os sintomas extraídos
    return symptoms


In [8]:
def generate_summary(symptoms):
    """
    Função para gerar um resumo dos sintomas usando um modelo generativo.

    Parâmetros:
    symptoms (str): String contendo os sintomas

    Retorna:
    symptoms_final (str): Resumo dos sintomas gerado pelo modelo
    """
    # Cria um prompt para o modelo generativo
    prompt = f'Summarize {symptoms} in just bulletpoints, without titles, about health symptoms with few words.'
    
    # Chama o modelo generativo para gerar um resumo
    response = model.generate_content(prompt)
    
    # Remove caracteres que não sejam letras ou espaços do resumo gerado
    symptoms_final = re.sub(r"[^a-zA-Z\s]", "", response.text)
    
    # Retorna o resumo dos sintomas final
    return symptoms_final



In [9]:
for disease in diseases:
    response = fetch_disease_info(disease)

    # Verifica se a resposta é válida (diferente de None ou False)
    if response and response.content:  # Verifica se há conteúdo na resposta
        symptoms = parse_symptoms(response.content)

        # Processa os sintomas apenas se não for uma string vazia
        if symptoms:  
            summary = generate_summary(symptoms)
            # Divide o resumo em uma lista de sintomas, removendo espaços em branco
            disease_symptoms[disease] = [item.strip() for item in summary.split('\n') if item.strip()]
        else:
            # Se não houver sintomas, armazena uma lista vazia
            disease_symptoms[disease] = []  
    else:
        # Se a resposta não for válida, armazena uma lista vazia
        disease_symptoms[disease] = []

    time.sleep(20)  # Aguarda 20 segundos para evitar sobrecarga no servidor


In [15]:
# Cria uma lista com as chaves do dicionário disease_symptoms cujos valores são listas vazias
empty_symptoms_keys = [key for key, value in disease_symptoms.items() if not value]

# Imprime o número de chaves com valores vazios
print(len(empty_symptoms_keys))


151


In [11]:
# Salvar o dicionário filtrado em um arquivo JSON
with open("empty_symptoms_who.json", "w") as json_file:
    json.dump(empty_symptoms_keys, json_file, indent=4)

In [12]:
# Filtrar doenças com sintomas não vazios
filtered_diseases_symptoms = {disease: symptoms for disease, symptoms in disease_symptoms.items() if symptoms}

In [13]:
# Contar o número de elementos no dicionário filtrado
num_elements = len(filtered_diseases_symptoms)
print("Número de elementos no dicionário filtrado:", num_elements)

Número de elementos no dicionário filtrado: 70


In [14]:
# Salvar o dicionário filtrado em um arquivo JSON
with open("diseases_symptoms_who.json", "w") as json_file:
    json.dump(filtered_diseases_symptoms, json_file, indent=4)